In [29]:
%pwd

'C:\\Users\\mathe\\Udemy\\Mlops_course\\section18\\data-science-project'

In [30]:
import os

In [31]:
os.chdir("C:/Users/mathe/Udemy/Mlops_course/section18/data-science-project")
%pwd

'C:\\Users\\mathe\\Udemy\\Mlops_course\\section18\\data-science-project'

In [32]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    model_name: str
    train_data: Path
    test_data: Path
    root_dir: Path
    alpha: float
    l1_ratio: float
    target_column: str

In [33]:
from src.DataScienceProject.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.DataScienceProject.utils.common import read_yaml, create_directories

In [34]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            model_name = config.model_name,
            train_data = config.train_data,
            test_data = config.test_data,
            root_dir = config.root_dir,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema
        )


        return model_trainer_config

In [35]:
from src.DataScienceProject import logger
from sklearn.linear_model import ElasticNet
import pandas as pd
import joblib

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train_model(self):
        try:
            logger.info("Carregando dados de treinamento")
            train_data = pd.read_csv(self.config.train_data)

            logger.info("Dividindo dados de treinamento em X e y")
            x_train = train_data.drop(columns=[self.config.target_column], axis=1)
            x_test = train_data.drop(columns=[self.config.target_column], axis=1)
            y_train = train_data[self.config.target_column]
            y_test = train_data[self.config.target_column]

            logger.info("Treino modelo ElasticNet")
            model = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
            model.fit(x_train, y_train)

            logger.info("Salvando modelo")
            model_path = os.path.join(self.config.root_dir, self.config.model_name)
            joblib.dump(model, model_path)

            logger.info("Modelo Salvo")
        except Exception as e:
            logger.exception(e)
            raise e

In [37]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train_model()

except Exception as e:
    logger.exception(e)
    raise e

[2026-02-06 06:09:02,322: INFO: common: YAML file: config\config.yaml carregado com sucesso:]
[2026-02-06 06:09:02,326: INFO: common: YAML file: params.yaml carregado com sucesso:]
[2026-02-06 06:09:02,326: INFO: common: YAML file: schema.yaml carregado com sucesso:]
[2026-02-06 06:09:02,326: INFO: common: Diretório criado em: artifacts:]
[2026-02-06 06:09:02,334: INFO: common: Diretório criado em: artifacts/model_trainer:]
[2026-02-06 06:09:02,334: INFO: 2428174529: Carregando dados de treinamento:]
[2026-02-06 06:09:02,360: INFO: 2428174529: Dividindo dados de treinamento em X e y:]
[2026-02-06 06:09:02,364: INFO: 2428174529: Treino modelo ElasticNet:]
[2026-02-06 06:09:02,368: INFO: 2428174529: Salvando modelo:]
